In [1]:
%load_ext autoreload

%autoreload 2

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
df_train = pd.read_csv('data/out_latent.csv').T

df_train_0 = df_train[df_train[9216] == 0][range(12*768)]
df_train_1 = df_train[df_train[9216] == 1][range(12*768)]

def compute_class_variables(df):

    mean_vectors = []
    for i, row in tqdm(df.iterrows()):
        mean_vector = np.mean(np.reshape(np.array(row), (768, 12)), axis = 1)
        mean_vectors.append(mean_vector)

    X = np.array(mean_vectors)
    cov_matrix = np.cov(X.T)
    precision_matrix = np.linalg.inv(cov_matrix)
    esperance_vector = np.mean(X, axis = 0)
    class_variables = (esperance_vector, cov_matrix)

    return class_variables, X

class_0_variables, vectors_0 = compute_class_variables(df_train_0)
class_1_variables, vectors_1 = compute_class_variables(df_train_1)

df_train_0 = None
df_train_1 = None

763it [00:00, 6416.69it/s]
1058it [00:00, 4793.54it/s]


In [4]:
from aiirw import AI_IRW

In [24]:
def D_IRW(x, prediction, vectors_0, vectors_1):
    if prediction == 0:
        train_vectors = vectors_0
    elif prediction == 1:
        train_vectors = vectors_1
    else:
        return ValueError
    
    return AI_IRW(X=train_vectors, AI=False, X_test=np.reshape(x, (1, 768)), n_dirs=1000)[0]


In [25]:
X = np.array(df_train)
v = X[0, :]

softmax = v[-1]
pred = v[-2]
v = np.reshape(v[:-2], (768, 12))

avg_v = np.mean(v, axis = 1)

print(D_IRW(avg_v, pred, vectors_0, vectors_1))


0.23060943643512452
